In [4]:
!pip install tensorflow-gpu==1.15
!pip install --upgrade gpt-2-simple
!pip install google-cloud-pubsub==1.4.2

Requirement already up-to-date: gpt-2-simple in /opt/conda/lib/python3.6/site-packages (0.7.1)
     |████████████████████████████████| 135 kB 1.4 MB/s 
  Created wheel for grpc-google-iam-v1: filename=grpc_google_iam_v1-0.12.3-py3-none-any.whl size=18499 sha256=0c368f64913528965f1c88987cb29668c06d61651800e4a0eef1fb8525033df8
  Stored in directory: /home/jovyan/.cache/pip/wheels/76/65/cd/392da05e43270f143b6c5076ba88d39144abff586792593e7c
Successfully built grpc-google-iam-v1


In [1]:
import gpt_2_simple as gpt2
import os
import requests

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [3]:
model_name = "124M"
if not os.path.isdir(os.path.join("models", model_name)):
    print(f"Downloading {model_name} model...")
    gpt2.download_gpt2(model_name=model_name)

Fetching checkpoint: 1.05Mit [00:00, 447Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 69.6Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 415Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:02, 169Mit/s]                                   
Fetching model.ckpt.index: 1.05Mit [00:00, 449Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 126Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 153Mit/s]                                                       


In [4]:
file_name = "../data/output_gpt.txt"

In [5]:
sess = gpt2.start_tf_sess()

In [6]:
gpt2.finetune(
    sess,
    dataset=file_name,
    model_name='124M',
    steps=1000,
    restore_from='fresh',
    run_name='run1',
    print_every=10,
    sample_every=200,
    save_every=500
)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:11<00:00, 11.54s/it]


dataset has 1721069 tokens
Training...
[10 | 343.53] loss=3.72 avg=3.72
interrupted
Saving checkpoint/run1/model-14


## Load the model

In [7]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

Loading checkpoint checkpoint/run1/model-1000
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-1000


In [13]:
gpt2.generate(sess, run_name='run1', prefix="<s>", length=160, nsamples=10, batch_size=10)

<s>this place is by far one of the best places ive been to in the city the service was exceptional and the food was on point i ordered the mushroom risotto and the salad and was shocked at how good it was the mushrooms were fresh and the risotto was perfectly seasoned and seasoned i even thought the portion wasnt huge but the size and number of servings were fantastic i highly recommend this place and i look forward to visiting again soon</s>
<s>so i just got off on a train from brooklyn and i walk right past this place i thought the decor was pretty cool but as i pass by this store i realize it has kinda a boring look to us barkeeps and employees but who cares the place is still pretty cool for real ill definitely come back and try the
<s>a few days ago i was visiting grandma and i decided to check out the britney cafe i dont remember much of it but i remember the decor and the ambiance very well the britneys isnt as big as it should be but i think it will be around or so it was a big

## Start Simulation

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/jovyan/config/gcloud-svc-acc/big-data-ca-svc-acc.json"

#### Utils

In [48]:
import string
import random
import time
import json

def generate_review_id(stringLength=22):
    """Inputs id length returns generated id

    Parameters
    ----------
    stringLength : int
        The length of id to generate

    Returns
    -------
    alphanumeric string
        a string of length stringLength
    """
    lettersAndDigits = string.ascii_letters + string.digits
    return "".join(random.choice(lettersAndDigits) for i in range(stringLength))

In [6]:
from google.cloud import pubsub_v1

project_id = "big-data-project-272506"
topic_name = "yelp-restaurant-reviews"

publisher = pubsub_v1.PublisherClient()
topic_path = publisher.topic_path(project_id, topic_name)

In [16]:
generated_reviews = gpt2.generate(sess, run_name='run1', prefix="<s>", length=160, nsamples=10, batch_size=10, return_as_list=True)

In [51]:
while True:
    print("Generating reviews...")
    generated_reviews = gpt2.generate(sess, run_name='run1', prefix="<s>", length=160, nsamples=10, batch_size=10, return_as_list=True)
    for reviews in generated_reviews:
        reviews_list = reviews.split('</s>')
        if len(reviews_list) > 1:
            review_content = reviews_list[0]
            review_content = review_content.replace("<s>", "")
            review = {
                "review_id": generate_review_id(),
                "reviewer_id": generate_review_id(),
                "restaurant_id": generate_review_id(),
                "timestamp": int(time.time()),
                "rating": random.randint(0,5),
                "review_content": review_content,
                "friend_count": random.randint(0, 50),
                "review_count": random.randint(0, 50),
                "useful_count": random.randint(0, 50),
                "cool_count": random.randint(0, 50),
                "funny_count": random.randint(0, 50),
                "restaurant_rating": random.randint(0,5)
            }
            data = json.dumps(review)
            data = data.encode("utf-8")
            future = publisher.publish(topic_path, data=data)
            print("Review published with id:", future.result())

Generating reviews
Review published with id: 1096844138157226
Review published with id: 1096843873373307
Review published with id: 1096844305763357
Review published with id: 1096843880747655
Generating reviews


KeyboardInterrupt: 